In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time
from moviepy.editor import VideoClip
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose

In [2]:
# Load the video file
video = cv2.VideoCapture('input.mp4')

# Get the first frame
ret, frame = video.read()
while video.isOpened():
    ret, frame = video.read()
    cv2.imshow('Mediapipe Feed', frame)
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)

    print("FPS: ",fps)
    print("Width: ", width)
    print("Height: ", height)
    break

        
video.release()
cv2.destroyAllWindows()

FPS:  25.0
Width:  854
Height:  640


In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        cv2.putText(image, str(round(angle, 2)), tuple(np.multiply(b, [640, 480]).astype(int)),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    return angle

In [ ]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('input.mp4')
# Curl counter variables
counter = 0 
fail=0
t0=0
t1=0
bt=0
st=0
is_last_frame = False
stage = None
alert =None
stat= None
t0_assigned=False
fps = 25
frames=[]
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            is_last_frame = True
            break
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    

        image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for leg closer to camera
            if (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z)>(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z):
                
                hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            else:
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate angle
            angle = calculate_angle(hip,knee,ankle)            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640, 854]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # Rep_counter

            if angle<140:
                stage="bent"
                bt=time.time()
                if not t0_assigned:
                    t0=time.time()
                    t1=t0+9
                    t0_assigned=True            
                if angle<80 and angle>30:
                    stat="Good"
                elif angle>80 and angle<120:
                    stat="Average"
                elif angle<30:
                    stat="low visibility"
                elif angle>=120 and angle<140:
                    stat="Poor"
            if angle>=140:
                stat=None
                
            if time.time()<t1:
                if stage=="bent":
                    alert="hold for "+str(int(abs(time.time()-t1)))+"s"
                if angle>=140:
                    alert="keep knee bent"
                    stage="straight"
                    st=time.time()
                    if abs(st-bt)<1.5:
                        alert="sudden change"
                    t0=time.time()
                    t1=t0+9
            if time.time()>=t1 and stage=="bent":
                alert="stretch your leg"
                if angle>=140:
                    stage="straight"
                    alert="bend again"
                    t0_assigned=False
                    st=time.time()
                    counter+=1
                    if abs(st-bt)>1.5:
                        alert="sudden change"
                        counter+=1
                    
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (800,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (150,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (150,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

        # Alerts 
        cv2.putText(image, 'ALERT', (350,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, alert, 
                    (350,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        # Stats 
        cv2.putText(image, 'STAT', (650,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stat, 
                    (650,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        frames.append(image)        
        
        if cv2.waitKey(10) & 0xFF == ord('q') or is_last_frame:
            print(stcntr,str(ft))
            break


# Save the video clip to a file
cap.release()
cv2.destroyAllWindows()
clip = VideoClip(lambda t: frames[int(t*fps)], duration=len(frames)/fps)
clip.write_videofile(f"output_{counter}_reps_.mp4", fps=fps)


In [13]:
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('input.mp4')

# Curl counter variables
counter = 0 
flag=0
fail=0
t0=0
t1=0
st=0
at=0
ct=0
ust=0
stcntr=0
stage = None
alert =None
stat= None
t0_assigned=False
fps = 25
frames=[]
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for leg closer to camera
            if (landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z)>(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z):
                
                hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            else:
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

            # Calculate angle
            angle = calculate_angle(hip,knee,ankle)            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640, 854]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            if angle < 80:
                stcntr+=1
                at=time.time()
            if angle>80:
                ft=ft+(time.time()-at)
            # Rep_counter

            if angle < 140 :
                ust=time.time()
                stage="BENT"
                if not t0_assigned:
                    t0 = time.time()
                    t1=t0+8
                    t0_assigned = True
#                     alert="Hold"
            if time.time()<t1 and stage=="BENT":
                alert="Hold"
                
            if stage=="BENT" and time.time()>=t1:
                alert="Stretch Your leg"
            
            elif (stage=="STRETCHED" and time.time()<t1):
                alert="Keep your knee bent"
                fail+=1
                t0=time.time()

            if angle > 140 and stage =='BENT' and time.time()-ust>2:
                stage="STRETCHED"
                alert=None

#                 st=time.time()
                if time.time()-t1>5:
                    counter +=1
                    ct=time.time()
                    alert="Bend your knee"
                    t0_assigned=False
            
            if fail>=5:
                stat="Poor"
            else:
                stat="Good"
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (700,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (150,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (150,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

        # Alerts 
        cv2.putText(image, 'ALERT', (350,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, alert, 
                    (350,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        # Stats 
        cv2.putText(image, 'STAT', (650,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stat, 
                    (650,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)
        frames.append(image)
        # Create a video clip from the frames
        clip = VideoClip(lambda t: frames[int(t*fps)], duration=len(frames)/fps)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            print(stcntr,str(ft))
            break


# Save the video clip to a file
cap.release()
# out.release()

cv2.destroyAllWindows()
clip.write_videofile("output_2_test.mp4", fps=fps)


QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread (0x31d7850).
Cannot move to target thread (0x22b3c20)

QObject::moveToThread: Current thread (0x22b3c20) is not the object's thread

t: 100%|███████████████████████████| 248/248 [13:06<00:00, 270.33it/s, now=None]

103 401811024298.2663
Moviepy - Building video output_2_test.mp4.
Moviepy - Writing video output_2_test.mp4




t: 100%|███████████████████████████| 248/248 [13:07<00:00, 270.33it/s, now=None]

Moviepy - Done !
Moviepy - video ready output_2_test.mp4
